- Il passaggio da dataset in formato *json* a *csv* è stato realizzato, considerando che il numero relativo all'Episode ID deve essere modificato in base al dialogo effettivo, infatti se una nuova sezione si apre con una frase citata precedentemente allora quella avrà lo stesso ID del dialogo precendete


In [ ]:
import json
import pandas as pd
from google.colab import drive
import os
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/dataset'

files = os.listdir(drive_path)

path_to_meld = 'drive/My Drive/MELD'

folder_path = 'MELD'

# Elenco dei file nella cartella
file_list = os.listdir(path_to_meld)
filtered_list = [f for f in file_list if f != '.ipynb_checkpoints']
print(file_list)

name_columns = ["Episode_Id", "Speaker", "Emotion_type", "Utterance", "Value(0/1)"]

for name in filtered_list:

  file_initials =  name
  #print(file_initials)

  if "test" in file_initials:
    is_test = True

  else:
    is_test = False

  file_name = path_to_meld  + "/" + file_initials

  with open(file_name, 'r') as file:
        dataset = json.load(file)

  episodes_list = [entry['episode'] for entry in dataset]
  speakers_list = [entry['speakers'] for entry in dataset]
  utterances_list = [entry['utterances'] for entry in dataset]
  print(utterances_list)
  emotions_list = [entry['emotions'] for entry in dataset]

  if is_test:
    triggers_list = [[0] * len(entry['speakers']) for entry in dataset]

  else:
    triggers_list = [entry['triggers'] for entry in dataset]

  data = [[], [], [], [], []]

# Questa è la parte in cui si modifica l'id del dialogo facendo in modo
# che se si incontrano utterances presenti anche in episodi precedenti
# e avranno lo stesso id perchè si riferiscono allo stesso discorso.

#un pattern ricorrente è che ci siano episodi che hanno in comune diverse frasi
#e che inziano sempre allo stesso modo.

  N = len(dataset)

  first_sentence = None
  id_dialog = -1

  for i in range(N):

    if first_sentence != utterances_list[i][0]:
        id_dialog  += 1
        first_sentence = utterances_list[i][0]

    for j in range(len(utterances_list[i])):
        data[0].append(id_dialog)
        data[1].append(speakers_list[i][j])
        data[2].append(emotions_list[i][j])
        data[3].append(utterances_list[i][j])
        data[4].append(triggers_list[i][j])

    for k in range(5):
        data[k].append("/")

#print(data[1])
  df = pd.DataFrame(data)
  df = df.transpose()
  df.columns = name_columns
  df.to_csv(file_initials[:-5] + ".csv", index=False)




Mounted at /content/drive
['MELD_val_efr.json', 'MELD_test_efr.json', 'MELD_train_efr.json']
[['Or! Or, we could go to the bank, close our accounts and cut them off at the source.', 'You\x92re a genius!', 'Aww, man, now we won\x92t be bank buddies!', 'Now, there\x92s two reasons.'], ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.', 'You\x92re a genius!', 'Aww, man, now we won\x92t be bank buddies!', 'Now, there\x92s two reasons.', 'Hey.', 'Hey!', 'Ohh, you guys, remember that cute client I told you about? I bit him.', 'Where?!', 'On the touchy.', 'And', 'No, I know!'], ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.', 'You\x92re a genius!', 'Aww, man, now we won\x92t be bank buddies!', 'Now, there\x92s two reasons.', 'Hey.', 'Hey!', 'Ohh, you guys, remember that cute client I told you about? I bit him.', 'Where?!', 'On the touchy.', 'And', 'No, I know!', 'I-I\x92m sorry, but the moment I touch him, I just wanna

JSONDecodeError: Invalid control character at: line 40685 column 9 (char 1007554)

- Poi bisogna eliminare i valori a nan e impostarli a 0.

In [ ]:
filtered_list_csv = [f[:-5] + ".csv" for f in file_list if f != '.ipynb_checkpoints']
print(filtered_list_csv)
num = 1

df_train = pd.read_csv(filtered_list_csv[0])
df_val = pd.read_csv(filtered_list_csv[1])
df_test = pd.read_csv(filtered_list_csv[2])

for df in [df_train, df_val, df_test]:
  if df.isnull().any().any():
    print(f"\nThere are NaN values in the column 'Value(0/1)' of the {num} dataset")
    df['Value(0/1)'] = df['Value(0/1)'].fillna(0)
    if df.isnull().any().any():
      print("There are NaN values in the column 'Value(0/1)' after transformation\n")
    else:
      print("There aren't NaN values in the column 'Value(0/1)' after transformation\n")

  else:
    print(f"\nThere aren't NaN values in the column 'Value(0/1)' of the {num} dataset")

  num+=1


- verificare e in caso ridistribuire tutto il dataset 80/10/10

Il dataset non è distributo con 80/10/10

In [ ]:
dim_tot = df_train.shape[0] + df_val.shape[0] + df_test.shape[0]
dim_train = round(0.80 * dim_tot)
print(dim_train)
dim_val_test = round(0.10 * dim_tot)
print(dim_val_test)
print(round(dim_train) + round(2*dim_val_test) == dim_tot)


### **DA RIVEDERE E MODIFICARE**

Non so bene il motivo ma potrebbe modificare il dataset iniziale json andando a eliminare/modificare alcune parti del testo

In [ ]:
import numpy as np

''' tot = [0] * len(df_train.columns)

for j in range(len(df_train.columns)):
  tot[j] = pd.concat([df_train.iloc[:,j].shape[0], df_val.iloc[:,j].shape[0], df_test.iloc[:,j].shape[0]])
  print(tot[j])

def resize_dataframe(df, target_dim):
    current_dim = len(df)
    if current_dim < target_dim:
        # Crea righe aggiuntive con zeri
        additional_rows = pd.DataFrame(0, index=np.arange(target_dim - current_dim), columns=df.columns)
        df = pd.concat([df, additional_rows], ignore_index=True)
    else:
        df = df.iloc[:target_dim, :]
    return df

df_train = resize_dataframe(df_train, dim_train)
df_val = resize_dataframe(df_val, dim_val_test)
df_test = resize_dataframe(df_test, dim_val_test)

for column,j in zip(df_train.columns, range(len(df_train.columns))):
  df_train[column] = tot[j].iloc[:dim_train].values
  df_val[column] = tot[j].iloc[dim_train:dim_train + dim_val_test].values
  df_test[column] = tot[j].iloc[dim_train + dim_val_test:].values


print(df_train.shape)
print(df_val.shape)
print(df_test.shape) '''

Quello che vogliamo però è che non ci siano stessi episodi contenuti in due dataframe quindi ogni df dovrà terminare con "/"

In [ ]:
print(df_train.iloc[42073, :])
print(df_val.iloc[-1, :])
print(df_test.iloc[-1, :])